In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm
import time

In [2]:
import rotation_matrix as rm

In [3]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [4]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [5]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/Assembly7_TCP.txt', delimiter=',', skiprows=1)

In [6]:
data.shape[0]

1168

In [7]:
# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254
data[:, 2] = data[:, 2] + 0.25307274

In [8]:
data

array([[-7.76358620e-05,  1.26373128e-04,  2.54739410e-01],
       [-1.55077668e-04,  2.52867414e-04,  2.56406070e-01],
       [-2.32289604e-04,  3.79506480e-04,  2.58072740e-01],
       ...,
       [-7.76358620e-05,  1.26373128e-04,  2.54739430e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.53072764e-01],
       [ 7.78476980e-05, -1.26239778e-04,  2.51406100e-01]])

In [9]:
start_pos = rtde_r.getActualTCPPose()

start_pos

[-0.0643954682958686,
 0.5262289800406665,
 0.2360106561536429,
 -1.3638071715629587,
 0.8763843439287416,
 0.8307796712233106]

In [10]:
start_pos = [-0.0555077809037947,
 0.5267063264795802,
 0.1640237148198558,
 -1.5588760353504112,
 0.03869269986900248,
 -0.023812772591892194]

In [11]:
rtde_c.moveL(start_pos, 0.1, 0.1)

True

In [12]:
init_q = rtde_r.getActualQ()
init_q

[2.2668793201446533,
 -1.667323728600973,
 -2.453174591064453,
 -2.153083940545553,
 -0.825841252003805,
 -3.1329835096942347]

In [13]:
init_q[5] += 0.25307274
rtde_c.moveJ(init_q, 0.1, 0.1)

True

In [14]:
start_pos_flask2 = rtde_r.getActualTCPPose()

In [15]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    #positions.append([data[i,0], data[i,1], 0.0, 0.0, 0.0, data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups
    positions.append([data[i,0], -data[i,1], 0.0, 0.0, 0.0, data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [16]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [17]:
positions_converted

[[-0.05558418235338805,
  0.526707915878114,
  0.1641508281034312,
  -1.5439514747779277,
  0.23679797896109792,
  0.17702450137141298],
 [-0.05566038878881079,
  0.5267094961176445,
  0.16427806064545714,
  -1.543793372509749,
  0.23809065035556468,
  0.17833745024370254],
 [-0.05573636420755403,
  0.5267110654928596,
  0.1644054357172194,
  -1.5436344822547658,
  0.23938327622862363,
  0.17965038491100174],
 [-0.055812132928530725,
  0.5267126252321225,
  0.1645329309497723,
  -1.5434748049272426,
  0.24067584866028233,
  0.18096329733127683],
 [-0.05588769396244486,
  0.5267141753281634,
  0.1646605435395587,
  -1.543314341460051,
  0.24196835973178618,
  0.18227617946299468],
 [-0.05596303812596699,
  0.5267157153568706,
  0.16478827847714222,
  -1.5431530898950638,
  0.2432608247894668,
  0.18358904689649316],
 [-0.05603817286819378,
  0.5267172454104438,
  0.16491615361456083,
  -1.5429910482360891,
  0.24455325917702417,
  0.1849019152209867],
 [-0.05611308822566128,
  0.5267187

In [18]:
# get first 5 x and z positions 
positions = positions_converted[0:15]

# only get x and z positions
positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*60)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

[0.00014830893617789965, 0.00014831275454139342, 0.00014831026060984975, 0.00014830523563974976, 0.0001483002267574724, 0.00014831480121455228, 0.0001483071493223282, 0.00014830771651909528, 0.00014831406394189743, 0.00014829745701060306, 0.00014831243607229654, 0.0001483132918663687, 0.00014830812447670197, 0.00014831215830644263]
[0.008898536170673979, 0.008898765272483605, 0.008898615636590985, 0.008898314138384985, 0.008898013605448344, 0.008898888072873137, 0.008898428959339693, 0.008898462991145717, 0.008898843836513846, 0.008897847420636184, 0.008898746164337792, 0.008898797511982123, 0.008898487468602117, 0.008898729498386559]
Speed in m/s:  0.008898534053385646


In [33]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif positions_converted[i] == positions_converted[i-1] == positions_converted[i-2] == positions_converted[i-3] == positions_converted[i-4]:
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

527
117


In [25]:
# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

In [26]:
# print length of both lists
print(len(positions_converted1))
print(len(positions_converted2))


527
641


In [35]:
velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.002
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(len(positions_converted1)):
    path.append([positions_converted1[i][0], positions_converted1[i][1], positions_converted1[i][2], positions_converted1[i][3], positions_converted1[i][4], positions_converted1[i][5], velocity, acceleration, blend_i])

rtde_c.moveL(path)

time.sleep(count/60)

path_2 = []
for i in range(len(positions_converted2)):
    path_2.append([positions_converted2[i][0], positions_converted2[i][1], positions_converted2[i][2], positions_converted2[i][3], positions_converted2[i][4], positions_converted2[i][5], velocity, acceleration, blend_i])

rtde_c.moveL(path_2)

rtde_c.stopScript()

In [71]:
velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.003
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(data.shape[0]-1):
    path.append([positions_converted[i][0], positions_converted[i][1], positions_converted[i][2], positions_converted[i][3], positions_converted[i][4], positions_converted[i][5], velocity, acceleration, blend_i])


#path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
#path.append(path_pose3)

rtde_c.moveL(path)

rtde_c.stopScript()

AttributeError: 'rtde_control.RTDEControlInterface' object has no attribute 'startScript'